In [19]:
# Importing libraries

import pandas as pd
import numpy as np
import urllib.request
from bs4 import BeautifulSoup
!pip install geocoder
!pip install folium
import geocoder
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [22]:
# Importing Wikipedia webpage
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")
our_table = soup

In [23]:
# Creating three seperate lists for each column and appending target values; creating the table
A = []
B = []
C = []

for row in our_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells) == 3:
        A.append(cells[0].find(text = True))
        B.append(cells[1].find(text = True))
        C.append(cells[2].find(text = True))

In [24]:
# Naming the columns
df = pd.DataFrame(A, columns = ['Postal Code'])
df['Borough'] = B
df['Neighbourhood'] = C

In [25]:
# Dropping 'Not assigned' values from the Borough column
# Cutting the \n values which would otherwise print as well
df.drop(df.loc[df['Borough'] == 'Not assigned\n'].index, inplace = True)
df = df.replace('\n','', regex = True)

In [26]:
# Merging the Neighbourhoods that have the same Postal Code
df.groupby('Postal Code')['Neighbourhood'].apply(','.join)

Postal Code
M1B                                       Malvern, Rouge
M1C               Rouge Hill, Port Union, Highland Creek
M1E                    Guildwood, Morningside, West Hill
M1G                                               Woburn
M1H                                            Cedarbrae
M1J                                  Scarborough Village
M1K          Kennedy Park, Ionview, East Birchmount Park
M1L                      Golden Mile, Clairlea, Oakridge
M1M      Cliffside, Cliffcrest, Scarborough Village West
M1N                          Birch Cliff, Cliffside West
M1P    Dorset Park, Wexford Heights, Scarborough Town...
M1R                                    Wexford, Maryvale
M1S                                            Agincourt
M1T              Clarks Corners, Tam O'Shanter, Sullivan
M1V    Milliken, Agincourt North, Steeles East, L'Amo...
M1W                        Steeles West, L'Amoreaux West
M1X                                          Upper Rouge
M2H                

In [27]:
# Replacing 'Not assigned' in Neighbourghoods with the value from Borough
df['Neighbourhood'].replace('Not assigned',df['Borough'])

2                                              Parkwoods
3                                       Victoria Village
4                              Regent Park, Harbourfront
5                       Lawrence Manor, Lawrence Heights
6            Queen's Park, Ontario Provincial Government
8                Islington Avenue, Humber Valley Village
9                                         Malvern, Rouge
11                                             Don Mills
12                       Parkview Hill, Woodbine Gardens
13                              Garden District, Ryerson
14                                             Glencairn
17     West Deane Park, Princess Gardens, Martin Grov...
18                Rouge Hill, Port Union, Highland Creek
20                                             Don Mills
21                                      Woodbine Heights
22                                        St. James Town
23                                    Humewood-Cedarvale
26     Eringate, Bloordale Gard

In [28]:
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [29]:
geo = pd.read_csv('http://cocl.us/Geospatial_data')
geo.tail()

,Postal Code,Latitude,Longitude
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437
102,M9W,43.706748,-79.594054


In [31]:
# Merging both tables together to add 'Latitude' and 'Longitude'
df2 = df.merge(geo, how = 'inner', on = 'Postal Code')

In [32]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [34]:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [42]:
toronto_data = df2[df2['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
toronto_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [43]:
CLIENT_ID = '2K10VTMNS55ZCVJWRIJ3H3V5FXAZC023DJ1ODSON1Y3AZFDF' # your Foursquare ID
CLIENT_SECRET = '2K10VTMNS55ZCVJWRIJ3H3V5FXAZC023DJ1ODSON1Y3AZFDF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2K10VTMNS55ZCVJWRIJ3H3V5FXAZC023DJ1ODSON1Y3AZFDF
CLIENT_SECRET:2K10VTMNS55ZCVJWRIJ3H3V5FXAZC023DJ1ODSON1Y3AZFDF


In [44]:
toronto_data.loc[0, 'Neighbourhood']

'Regent Park, Harbourfront'

In [46]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


In [58]:
radius = 500
limit = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, 
CLIENT_SECRET, VERSION, neighborhood_latitude, neighborhood_longitude,radius,LIMIT)